In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from imp_lib import *
import scipy.ndimage as snd
import functools

#Code căn chỉnh lại ảnh
%matplotlib qt5
img = cv2.imread("Images/o_to_1.png")
img_copy = np.copy(img)
img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2GRAY)
#plt.imshow(img_copy)

#get 4 points at corners 
pt_A = [44,162]
pt_B = [51,213]
pt_C = [200,168]
pt_D = [191,128]

# Here, I have used L2 norm. You can use L1 also.
width_AD = np.sqrt(((pt_A[0] - pt_D[0]) ** 2) + ((pt_A[1] - pt_D[1]) ** 2))
width_BC = np.sqrt(((pt_B[0] - pt_C[0]) ** 2) + ((pt_B[1] - pt_C[1]) ** 2))
maxWidth = max(int(width_AD), int(width_BC))
 
 
height_AB = np.sqrt(((pt_A[0] - pt_B[0]) ** 2) + ((pt_A[1] - pt_B[1]) ** 2))
height_CD = np.sqrt(((pt_C[0] - pt_D[0]) ** 2) + ((pt_C[1] - pt_D[1]) ** 2))
maxHeight = max(int(height_AB), int(height_CD))


input_pts = np.float32([pt_A, pt_B, pt_C, pt_D])
output_pts = np.float32([[0, 0],
                        [0, maxHeight - 1],
                        [maxWidth - 1, maxHeight - 1],
                        [maxWidth - 1, 0]])
M = cv2.getPerspectiveTransform(input_pts,output_pts)
out = cv2.warpPerspective(img_copy,M,(maxWidth, maxHeight))

#imgs = [img, out]
#titles = ['Ảnh gốc', 'Ảnh biến đổi']
#MultipleImages(imgs,titles,figsize=[20,5],ncols=2)

#Đặt ngưỡng
def AdaptiveThreshold(img, blockSize, C):
    # Mở file và chuyển sang ảnh xám
    a = np.asarray(img)
    # Phân đoạn bằng adaptive threshold 
    b = cv2.adaptiveThreshold(a,a.max(),
	    cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,blockSize,C)

    return b

b = AdaptiveThreshold(out, blockSize=51, C=10)


# detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
_,contours,_ = cv2.findContours(b, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                                      
# draw contours on the original image
image_b = cv2.cvtColor(b,cv2.COLOR_GRAY2BGR)
image_copy = image_b.copy()
boundingBoxes = [cv2.boundingRect(c) for c in contours]
ROI_number = 0

template = cv2.imread('8.png')
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template_gray = cv2.bitwise_not(template_gray)
width = 0
height = 0
scale =[1,1]
tmp = [template_gray.shape[0],template_gray.shape[1]]

threshold = 0.5
pad_number = 50 #padding roi array

for rect in boundingBoxes:
    x,y,w,h = rect
    if w>0 and h>40 :
        #cv2.rectangle(image_copy, (x,y), (x+w,y+h), (0, 0, 255), 2)
        ROI = b[y:y+h, x:x+w]
        
        scale[0] = h/tmp[0]
        scale[1] = w/tmp[1]
        width = (int)(tmp[1]*scale[1])
        height = (int)(tmp[0]*scale[0])
        #print("height:",height-h)
        #print("width :",width-w)
        template_gray = cv2.resize(template_gray,(width, height))
        #cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
        #ROI_number += 1
        ROI = np.pad(ROI,[(0, pad_number),(0, pad_number)],constant_values=(0, 0))
        res = cv2.matchTemplate(ROI,template_gray, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)
        print("Max_val :", max_val)
        if max_val > threshold :
            top_left = max_loc
            bottom_right = [top_left[0]+w, top_left[1]+h]
            cv2.rectangle(image_copy,(top_left[0]+x,top_left[1]+y), (bottom_right[0]+x,bottom_right[1]+y),(0, 0, 255), 2)
cv2.imwrite('segmentation.png',image_copy)       
        
#cv2.drawContours(image_copy, contours, -1, (255, 255, 0), 1)
#cv2.imshow("show", image_copy)
cv2.circle(img, (pt_A[0],pt_A[1]), radius=2, color=(255, 0, 0), thickness=-1)
cv2.circle(img, (pt_B[0],pt_B[1]), radius=2, color=(255, 0, 0), thickness=-1)
cv2.circle(img, (pt_C[0],pt_C[1]), radius=2, color=(255, 0, 0), thickness=-1)
cv2.circle(img, (pt_D[0],pt_D[1]), radius=2, color=(255, 0, 0), thickness=-1)
imgs = [img, image_copy]
titles = ['', '']
MultipleImages(imgs,titles,figsize=[20,5],ncols=2)
#cv2.imwrite("show.jpg", image_copy)
#cv2.imshow('anh cat',crop)

Max_val : 0.6233231425285339
Max_val : 0.1443692147731781
Max_val : 0.12761534750461578
Max_val : 0.15891140699386597
Max_val : 0.5534074902534485
Max_val : 0.186929851770401
Max_val : 0.22974678874015808
Max_val : 0.41891059279441833
